In [5]:
%load_ext autoreload
%autoreload 2

import logging
import gradio as gr

from agent.nodes_and_tools import create_agent_executor, stream_agent_output
from config.utils import setup_logging, get_logger
from config.config import ConfigManager

# checking where we are
!pwd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/Users/eliza/search-pydantic-ai


## Config and logging

In [6]:
setup_logging(level=logging.WARNING)
logger = get_logger(__name__)
logger.warning("Starting the application...")
logger.info("Starting the application...")
config = ConfigManager()

print(config.documentation_urls)
print(config.config['embed_with_openai'])

2025-05-13 19:03:53,253 - __main__ - WARNING - Starting the application...
['https://ai.pydantic.dev/']
True


## Agent interface

**Scroll below for the interactive interface**

In [7]:
agent_executor = create_agent_executor()

In [12]:
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=320)
    # Text input with stop button on right
    with gr.Row():
        with gr.Column(scale=20):
            msg = gr.Textbox(label="Message", placeholder="Ask something...")
            
        with gr.Column(scale=1, min_width=40):
            stop_btn = gr.Button("⏹️", size="sm", variant="stop")
    
    # Buttons below the text input        
    with gr.Row():
        send_btn = gr.Button("Send", size="md", variant="primary", scale=3)
        clear = gr.Button("Clear", size="md", scale=1)
    
    def respond(message, chat_history):
        # Add user message to chat history
        chat_history.append((message, ""))
        yield chat_history
        
        # Stream the agent's response
        bot_message = ""
        for chunk in stream_agent_output(agent_executor, message):
            bot_message += chunk
            # Update just the last bot message
            chat_history[-1] = (message, bot_message)
            yield chat_history
    
    # Set up events with cancellation
    msg_event = msg.submit(respond, [msg, chatbot], [chatbot])
    send_event = send_btn.click(respond, [msg, chatbot], [chatbot])
    
    # Clear button
    clear.click(lambda: None, None, chatbot, queue=False)
    
    # Stop button cancels both the message submit and send button events
    stop_btn.click(fn=None, inputs=None, outputs=None, cancels=[msg_event, send_event])

demo.launch(inline=True, share=False)

/var/folders/3n/7m69d59d44x4c5xxm3gwg9880000gn/T/ipykernel_14339/2156224731.py:2: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=320)


* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


/Users/eliza/search-pydantic-ai/venv/lib/python3.12/site-packages/networkx/readwrite/json_graph/node_link.py:287: FutureWarning: 
The default value will be changed to `edges="edges" in NetworkX 3.6.

To make this warning go away, explicitly set the edges kwarg, e.g.:

  nx.node_link_graph(data, edges="links") to preserve current behavior, or
  nx.node_link_graph(data, edges="edges") for forward compatibility.
  warnings.warn(
